In [1]:
import numpy as np
import pandas as pd
#train data
revenue_train=pd.read_csv('patient_monthwise_revenue_train.csv')
diagnosis_train=pd.read_csv('physio_diagnosis_train.csv')
classified_train=pd.read_csv('patient_train_classified.csv')
appts_train=pd.read_csv('physio_appts_train.csv')
#test data
revenue_test=pd.read_csv('patient_monthwise_revenue_test.csv')
diagnosis_test=pd.read_csv('physio_diagnosis_test.csv')
submission=pd.read_csv('Submission.csv')

/Users/admin/softs/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/admin/softs/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def avg_nps_val(val1 , val2):
    if is_number(val1):
        return val1
    if is_number(val2):
        return val2
    return 0.0

def diag_val(diag,nps,un):
    val=str(diag)
    if not nps.isdigit() and not nps=='nan':
        val=val+' '+ nps
    if not un.isdigit() and not un=='nan':
        val=val+' '+un
    return val

def get_month(val):
    return int(str(val).strip()[:2])

def get_year(val):
    return int(str(val).strip()[3:])

In [3]:
print list(revenue_train)
print list(revenue_test)

['patient_id', 'visit_month_year', 'service_id', 'visits_count', 'city', 'revenue', 'ref_type', 'ref_name', 'ref_source', 'service_name', 'FVD', 'FVM', 'FVS', 'approx_age', 'gender', 'LVD', 'brand', 'visits_required', 'diagnosis', 'avg_nps', 'Unnamed: 20', 'Unnamed: 21']
['patient_id', 'visit_month_year', 'service_id', 'city', 'ref_type', 'ref_name', 'ref_source', 'service_name', 'FVD', 'FVM', 'FVS', 'approx_age', 'gender', 'LVD', 'brand', 'visits_required', 'diagnosis', 'avg_nps', 'Unnamed: 18', 'Unnamed: 19']


In [4]:
#revenue_train[['FVD', 'FVM', 'FVS','LVD']].head()
revenue_train['is_recurring']=revenue_train.apply(lambda row: 1 if str(row['FVD'])==str(row['LVD']) else 0, axis=1)
revenue_test['is_recurring']=revenue_train.apply(lambda row: 1 if str(row['FVD'])==str(row['LVD']) else 0, axis=1)

In [5]:
revenue_train.drop(['FVD', 'FVM', 'FVS','LVD'],axis = 1, inplace=True)
revenue_test.drop(['FVD', 'FVM', 'FVS','LVD'],axis = 1, inplace=True)

In [6]:
for col in revenue_train.columns.values:
    if revenue_train[col].dtypes=='object':
        revenue_train[col]=revenue_train[col].str.replace('\N','')
        
for col in revenue_test.columns.values:
    if revenue_test[col].dtypes=='object':
        revenue_test[col]=revenue_test[col].str.replace('\N','')

In [7]:
for col in revenue_train.columns.values:
    if revenue_train[col].dtypes=='object':
        revenue_train[col]=revenue_train[col].str.replace('\\','')
        
for col in revenue_test.columns.values:
    if revenue_test[col].dtypes=='object':
        revenue_test[col]=revenue_test[col].str.replace('\\','')

In [8]:
revenue_train['diagnosis']=revenue_train.apply(lambda row:diag_val(row['diagnosis'],str(row['avg_nps']),str(row['Unnamed: 20'])), axis=1)
revenue_test['diagnosis']=revenue_test.apply(lambda row:diag_val(row['diagnosis'],str(row['avg_nps']),str(row['Unnamed: 18'])), axis=1)

In [9]:
revenue_train['avg_nps']=revenue_train.apply(lambda row: avg_nps_val(row['avg_nps'],row['Unnamed: 20']), axis=1)
revenue_test['avg_nps']=revenue_test.apply(lambda row: avg_nps_val(row['avg_nps'],row['Unnamed: 18']), axis=1)

In [10]:
revenue_train['avg_nps']=revenue_train['avg_nps'].astype('float')
revenue_test['avg_nps']=revenue_test['avg_nps'].astype('float')

In [11]:
revenue_train['avg_nps']=revenue_train['avg_nps'].fillna(0.0)
revenue_test['avg_nps']=revenue_test['avg_nps'].fillna(0.0)

In [12]:
revenue_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83307 entries, 0 to 83306
Data columns (total 19 columns):
patient_id          83307 non-null int64
visit_month_year    83307 non-null object
service_id          83307 non-null int64
visits_count        83307 non-null int64
city                83307 non-null object
revenue             83307 non-null float64
ref_type            83307 non-null object
ref_name            83307 non-null object
ref_source          83307 non-null object
service_name        82084 non-null object
approx_age          83307 non-null object
gender              83274 non-null object
brand               83307 non-null object
visits_required     83307 non-null int64
diagnosis           83307 non-null object
avg_nps             83307 non-null float64
Unnamed: 20         36 non-null object
Unnamed: 21         16 non-null object
is_recurring        83307 non-null int64
dtypes: float64(2), int64(5), object(12)
memory usage: 12.1+ MB


In [13]:
revenue_train.drop([ 'Unnamed: 20', 'Unnamed: 21'],axis = 1, inplace=True)
revenue_test.drop(['Unnamed: 18', 'Unnamed: 19'],axis = 1, inplace=True)

In [14]:
revenue_train['visit_month']=revenue_train['visit_month_year'].apply(lambda x: get_month(x))
revenue_test['visit_month']=revenue_test['visit_month_year'].apply(lambda x: get_month(x))
revenue_train['visit_year']=revenue_train['visit_month_year'].apply(lambda x: get_year(x) )
revenue_test['visit_year']=revenue_test['visit_month_year'].apply(lambda x: get_year(x))

In [15]:
revenue_train['visit_month']=revenue_train['visit_month'].astype('int')
revenue_test['visit_month']=revenue_test['visit_month'].astype('int')
revenue_train['visit_year']=revenue_train['visit_year'].astype('int')
revenue_test['visit_year']=revenue_test['visit_year'].astype('int')

In [16]:
revenue_train.drop(['visit_month_year'],axis = 1, inplace=True)
revenue_test.drop(['visit_month_year'],axis = 1, inplace=True)

In [17]:
#ref_type,visit_month_year
revenue_train['approx_age']=revenue_train['approx_age'].apply(lambda x: x if is_number(x) else None)
revenue_train['approx_age']=revenue_train['approx_age'].astype('float')

In [18]:
revenue_test['approx_age']=revenue_test['approx_age'].apply(lambda x: x if is_number(x) else None)
revenue_test['approx_age']=revenue_test['approx_age'].astype('float')

In [19]:
mean=revenue_train['approx_age'].mean()
revenue_train['approx_age']=revenue_train['approx_age'].fillna(mean)
mean=revenue_test['approx_age'].mean()
revenue_test['approx_age']=revenue_test['approx_age'].fillna(mean)

In [20]:
revenue_test['is_male']=revenue_test['gender'].apply(lambda x: 2 if str(x)=='male' else 1)
revenue_train['is_male']=revenue_train['gender'].apply(lambda x: 2 if str(x)=='male' else 1)

In [21]:
revenue_train.drop(['gender'],axis = 1, inplace=True)
revenue_test.drop(['gender'],axis = 1, inplace=True)

In [22]:
revenue_test['is_ref_type_b2c']=revenue_test['ref_type'].apply(lambda x: 2 if str(x)=='B2C' else 1)
revenue_train['is_ref_type_b2c']=revenue_train['ref_type'].apply(lambda x: 2 if str(x)=='B2C' else 1)

In [23]:
revenue_train.drop(['ref_type'],axis = 1, inplace=True)
revenue_test.drop(['ref_type'],axis = 1, inplace=True)

In [24]:
revenue_train['diagnosis']=revenue_train['diagnosis'].astype('string')
revenue_train['word_count'] = revenue_train['diagnosis'].apply(lambda x: len(str(x).split(" ")))
revenue_test['diagnosis']=revenue_test['diagnosis'].astype('string')
revenue_test['word_count'] = revenue_test['diagnosis'].apply(lambda x: len(str(x).split(" ")))

In [25]:
revenue_train['char_count'] = revenue_train['diagnosis'].str.len()
revenue_test['char_count'] = revenue_test['diagnosis'].str.len()

In [26]:
def avg_word(sentence):
    try:
        words = sentence.split()
        return (sum(len(word) for word in words)/len(words))
    except:
        return 1

revenue_train['avg_word'] = revenue_train['diagnosis'].apply(lambda x: avg_word(x))
revenue_test['avg_word'] = revenue_test['diagnosis'].apply(lambda x: avg_word(x))

In [27]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
revenue_train['stopwords'] = revenue_train['diagnosis'].apply(lambda x: len([x for x in x.split() if x in stop]))
revenue_test['stopwords'] = revenue_test['diagnosis'].apply(lambda x: len([x for x in x.split() if x in stop]))

/Users/admin/softs/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/admin/softs/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  after removing the cwd from sys.path.


In [28]:
revenue_train['numerics'] = revenue_train['diagnosis'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
revenue_test['numerics'] = revenue_test['diagnosis'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

In [29]:
revenue_train['upper'] = revenue_train['diagnosis'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
revenue_test['upper'] = revenue_test['diagnosis'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [30]:
revenue_train['diagnosis'] = revenue_train['diagnosis'].apply(lambda x: " ".join(x.lower() for x in x.split()))
revenue_test['diagnosis'] = revenue_test['diagnosis'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [31]:
revenue_train['diagnosis'] = revenue_train['diagnosis'].str.replace('[^\w\s]','')
revenue_test['diagnosis'] = revenue_test['diagnosis'].str.replace('[^\w\s]','')

In [32]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
revenue_train['diagnosis'] = revenue_train['diagnosis'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
revenue_test['diagnosis'] = revenue_test['diagnosis'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [33]:
from textblob import Word
revenue_train['diagnosis'] = revenue_train['diagnosis'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
revenue_test['diagnosis'] = revenue_test['diagnosis'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [34]:
revenue_train.drop(['visits_count', 'revenue'],axis = 1, inplace=True)

In [35]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for col in revenue_train.columns.values:
    if revenue_train[col].dtypes=='object':
        data=revenue_train[col].append(revenue_test[col])
        le.fit(data.values)
        revenue_train[col]=le.transform(revenue_train[col])
        revenue_test[col]=le.transform(revenue_test[col])

/Users/admin/softs/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:216: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
/Users/admin/softs/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:275: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]


In [36]:
classified_train.rename(columns={'PID': 'patient_id'}, inplace=True)

In [37]:
train_data=pd.merge(revenue_train,classified_train,how='left',on='patient_id')

In [38]:
y_val=train_data[['patient_id','Bucket','Revenue']]

In [39]:
train_data.drop(['patient_id','Bucket','Revenue'],axis = 1, inplace=True)

In [40]:
pat_ids=revenue_test['patient_id']
test_data=revenue_test
test_data.drop(['patient_id'],axis = 1, inplace=True)

In [41]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83307 entries, 0 to 83306
Data columns (total 21 columns):
service_id         83307 non-null int64
city               83307 non-null int64
ref_name           83307 non-null int64
ref_source         83307 non-null int64
service_name       83307 non-null int64
approx_age         83307 non-null float64
brand              83307 non-null int64
visits_required    83307 non-null int64
diagnosis          83307 non-null int64
avg_nps            83307 non-null float64
is_recurring       83307 non-null int64
visit_month        83307 non-null int64
visit_year         83307 non-null int64
is_male            83307 non-null int64
is_ref_type_b2c    83307 non-null int64
word_count         83307 non-null int64
char_count         83307 non-null int64
avg_word           83307 non-null int64
stopwords          83307 non-null int64
numerics           83307 non-null int64
upper              83307 non-null int64
dtypes: float64(2), int64(19)
memory usage: 14.

In [42]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


def random_forest(train_x,train_y,test_x,test_y):
    clf = RandomForestClassifier()
    clf.fit(train_x, train_y)
    prediction=clf.predict(test_x)
    print accuracy_score(test_y,prediction)
    return clf

In [43]:
def xgboost(train_x, train_y, test_x, test_y):
    clf=xgb.XGBClassifier(max_depth=5, n_estimators=300, learning_rate=0.03)
    clf.fit(train_x, train_y)
    prediction = clf.predict(test_x)
    print accuracy_score(test_y,prediction)
    return clf

In [42]:
submission.drop(['Bucket'],axis = 1, inplace=True)

In [47]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y =train_test_split(train_data, y_val['Bucket'], test_size=0.2, random_state=12)

In [49]:
rf_clf=xgboost(train_x,train_y,test_x,test_y)

0.685631976954


In [50]:
f_clf=random_forest(train_x,train_y,test_x,test_y)

0.867302844797


In [55]:
res=f_clf.predict(test_data)

In [56]:
sub = pd.DataFrame({ 'PID': pat_ids,'Bucket': res })

In [57]:
grouped = sub.groupby('PID')
p_id=[]
bucket=[]
for name, group in grouped:
   # print(name)
    low=0
    mid=0
    high=0
    high_med=0
    for val in group['Bucket']:
        if str(val)=="Low":
            low=low+1
        elif str(val)=="Med":
            mid=mid+1
        elif str(val)=="High":
            high=high+1
        else:
            high_med=high_med+1
    
    p_id.append(name) 
    
    if high>0:
        bucket.append("High")
    elif high_med>0:
        bucket.append("High-Med")
    elif mid>0:
        bucket.append("Med")
    else:
        bucket.append("Low")

In [58]:
subm = pd.DataFrame({ 'PID': p_id,'Bucket': bucket })
final=pd.merge(submission,subm,how='left',on='PID')
final[['PID','Bucket']].to_csv('sub.csv', index=False)

In [72]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [44]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y =train_test_split(train_data, y_val['Revenue'], test_size=0.2, random_state=12)

In [45]:
lreg = LinearRegression()

In [73]:
logisticRegr = LogisticRegression()

In [46]:
#T_train_xgb = xgb.DMatrix(train_x, train_y)
#params = {"objective": "reg:linear", "booster":"gblinear"}
#gbm = xgb.train(dtrain=T_train_xgb,params=params)
#res = gbm.predict(xgb.DMatrix(test_data))
#print res

In [47]:
#pred = lreg.predict(test_x)

In [48]:
#mse = np.mean((pred - test_y)**2)
#print mse

In [49]:
poly = PolynomialFeatures(2)
X = poly.fit_transform(train_data)
T = poly.fit_transform(test_data)

In [52]:
T.shape

(55329, 253)

In [53]:
Y= y_val['Revenue'].apply(np.int64)

In [54]:
clf = Perceptron(fit_intercept=False, max_iter=10, tol=None,shuffle=False).fit(X,Y)

In [66]:
lreg.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [ ]:
logisticRegr.fit(X,Y)

In [67]:
res=logisticRegr.predict(T)

In [68]:
sub = pd.DataFrame({ 'PID': pat_ids,'Bucket': res })

In [69]:
sub['Bucket'].value_counts()

-3681.766624     7
-2137.551542     5
-2178.879860     5
-1814.883375     4
 2088.907418     3
-5817.003354     3
-8018.781694     3
 4055.053200     3
-3997.711580     3
 3079.935683     2
 28901.454086    2
 6889.443776     2
 3439.977951     2
 6058.834574     2
 13067.887736    2
 1650.421651     2
 7242.754243     2
 10109.772668    2
 1363.461855     2
-2548.436180     2
-1750.667666     2
-1096.007622     2
 3395.196300     2
 18456.197975    2
 17134.449018    2
 3926.267012     2
-5364.294597     2
 18521.015521    2
-6837.359181     2
-2663.510474     2
                ..
 14661.858895    1
 57897.625120    1
 15197.699278    1
 7793.229749     1
-681.782581      1
 16233.577240    1
 21537.374201    1
 1603.697440     1
 3536.808150     1
 19502.779764    1
-7595.671203     1
 56686.178258    1
-2643.107128     1
 43479.040337    1
 3509.309232     1
 11820.226318    1
 16059.206964    1
 3685.292385     1
 3761.158720     1
 16079.122122    1
 32815.261784    1
 3711.292714

In [70]:
grouped = sub.groupby('PID')
p_id=[]
bucket=[]
for name, group in grouped:
   # print(name)
    low=0
    mid=0
    high=0
    high_med=0
    for val in group['Bucket']:
        if val >10000:
            high=high+1
        elif val>5000:
            high_med=high_med+1 
        elif val>1000:
            mid=mid+1
        else: 
            low=low+1
    
    p_id.append(name) 
    
    if high>0:
        bucket.append("High")
    elif high_med>0:
        bucket.append("High-Med")
    elif mid>0:
        bucket.append("Med")
    else:
        bucket.append("Low")

In [71]:
subm = pd.DataFrame({ 'PID': p_id,'Bucket': bucket })
final=pd.merge(submission,subm,how='left',on='PID')
final[['PID','Bucket']].to_csv('sub.csv', index=False)

In [ ]:
    '''if high>=low and high>=mid and high>= high_med:
        bucket.append("High")
    elif high_med>=low and high_med>=mid and high_med>=high:
        bucket.append("High-Med")
    elif mid>=low and mid>=high_med and mid>=high :
        bucket.append("Med")
    else:
        bucket.append("Low")'''